In [64]:
import copy

from board3 import Board3, sqr_distance
from controller3 import ActionController, MW_CELLS
import time
from heapq import heappush, heappop
from tqdm.auto import tqdm
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import concurrent.futures
import math
from collections import Counter, deque
from mcts import search
from nnl import to_emb, emb_mem
import pickle
from torch.distributions.categorical import Categorical

import copy


In [258]:
def select_greedy(policy, state):
    if len(ActionController(state).get_available_moves()) == 0:
        return 0
    return max(ActionController(state).get_available_moves(), key=lambda x: policy.get((state, x), 0.5))

bm = {}

for e in tqdm(range(20000)):
    eps = max(0.1, 1 - e / 20000)
    b = Board3(walk_time=1000)

    for i in range(100):
        bc = b.copy()
        nc = ActionController(bc)
        if random.random() < eps:
            a = random.choice(nc.get_available_moves())
        else:
            a = select_greedy(bm, bc)

        nc.execute_action(a)
        bc.step(1600)

        reward = 0
        if nc.is_win():
            reward = 1
        elif nc.is_lose():
            reward = -1
        elif nc.is_block():
            reward = -1

        end = reward != 0

        if (b, a) not in bm:
            bm[(b.copy(), a)] = 0.5

        if end:
            target = reward
        else:
            mv = max([bm.get((bc, q), 0.5) for q in range(ActionController.get_action_space())])
            target = reward + 0.9 * mv

        bm[(b.copy(), a)] = bm[(b, a)] + 0.2 * (target - bm[(b, a)])
        b = bc

        if end:
            break



  0%|          | 0/20000 [00:00<?, ?it/s]

In [259]:
print(len(bm))

658629


In [260]:
b = Board3(walk_time=1000, walk_frodo=False)
b.players_positions = [(2, 0), (0, 2), (3, 3)]

print(b)

mv = select_greedy(bm, b)

print(ActionController.explain_action(mv))

.  %  E  %
.  %  .  %
P  .  .  .
.  .  .  T
Skip


In [261]:


def eval_policy(pol, n=100):

    wins = 0
    loses = 0
    draws = 0

    for _ in range(n):
        b = Board3(walk_time=1000, walk_frodo=False)

        for i in range(100):
            bc = b.copy()
            nc = ActionController(bc)
            a = select_greedy(pol, bc)
            nc.execute_action(a)
            bc.step(1600)

            if nc.is_win():
                wins += 1
                break
            elif nc.is_lose():
                loses += 1
                break
            elif nc.is_block():
                draws += 1
                break

            b = bc

        else:
            draws += 1

    return wins / n, loses / n, draws / n


print(eval_policy(bm))


(0.07, 0.0, 0.93)
